### GitHub Repository

Browse it here https://github.com/immutable/StarkNetCourseContent

### About Jupyter Notebooks

A *Jupyter Notebook* can be used either as a static reference, where you can look at commands and their outputs, or as a dynamic playground, where you interact with the contents and execute the notebook to check results.

Notebooks are useful because they provide both theory, explanations and **executable** code in a single place.

You can view the notebook as a static resource in [GitHub][gh-notebook] itself.

[gh-notebook]: https://github.com/immutable/StarkNetCourseContent/blob/feat/proxy-and-oracle/src/StarkNet%20Course%20Week%208.ipynb

To interact with it, you will need to download the repository and make sure you have a proper setup locally.

### Running Notebooks locally

To execute notebooks locally, check out the [Setup][setup] documentation in the _README.md_ for the GitHub repository of the course.

You will then need to [install Jupyter][jup]. After that you can start up the course notebooks by running:

```sh
git clone git@github.com:immutable/StarkNetCourseContent
cd StarkNetCourseContent
jupyter notebook
```

Browse the `notebooks/` directory and find the one you are interested in.

[setup]: https://github.com/immutable/StarkNetCourseContent#setup
[jup]: https://jupyter.org/install#jupyter-notebook

# StarkNet Course: Week 8

This week we are using oracles as a case study for StarkNet smart contracts. Through this notebook, you will:

- Prepare your _StarkNet_ wallet.
- Create an oracle contract, step by step.
- Execute tests to make sure the contract conforms to the specifications.
- Deploy the contract.
- Interact with the deployed contract using the wallet.

# Creating your Wallet

In [20]:
%%bash

# REPLACE!! with your venv from the `setup` step above
source ~/cairo38_venv/bin/activate
# The kind of contract we use for our account
export STARKNET_WALLET=starkware.starknet.wallets.open_zeppelin.OpenZeppelinAccount
# Create an wallet for us
starknet deploy_account --network alpha-goerli

Sent deploy account contract transaction.

NOTE: This is a modified version of the OpenZeppelin account contract. The signature is computed
differently.

Contract address: 0x004765409f0dd450fa303e5f7cebfb1304b2f495b5ce940da8a5aa779018a412
Public key: 0x0283bf6ee82c97f7a0edc14f27f54ebedcdd48be64fc74ac127224fd03a8bbb0
Transaction hash: 0x20a075beee7a3de1b28678f120918e91e9971898cd575285bd3df95b9bbcd1d



# Adding Funds to your Wallet

To add funds, use the [StarkNet Faucet][faucet] with the address generated above (i.e.: the _Contract Address_).

[faucet]: https://faucet.goerli.starknet.io/

# Implementing an Oracle Contract

### The `%%file` Magic

You will notice we use `%%file` or `%%file -a` in several places below. It simply means "overwrite the file specified with the contents below" or for `%%file -a` "append the below to the specified file".

> TL;DR: each code box below appends to a file, so after executing all of them we will have a finished contract file.

### Imports

Like any good program, we start out by importing some external code to make our lives easier.

- `HashBuiltin` to read and write to the contract storage.
- `get_block_number` to use as timestamps for our oracle data points.
- `assert_lt` and `unsigned_div_rem` to process and validate inputs.

And also, `Ownable` from [OpenZeppelin][oz] so we can make sure our contract has proper access control.

[oz]: https://docs.openzeppelin.com/contracts-cairo/

In [115]:
%%file "../src/Oracle.cairo"

%lang starknet

// From the Cairo "standard library"
from starkware.cairo.common.cairo_builtins import HashBuiltin
from starkware.starknet.common.syscalls import get_block_number
from starkware.cairo.common.math import assert_lt, unsigned_div_rem

// From OpenZeppelin!
from openzeppelin.access.ownable.library import Ownable

Overwriting ../src/Oracle.cairo


### Types of Oracles

There are two major uses for oracles in the blockchain world:

- Timeseries data, usually for prices. This is usually the default interpretation for "oracle".
- Verifiable Random Functions (VRF): Hashes that can be proved to come from some input. Extremely useful for randomness in contracts.

### Storing Timeseries Data in StarkNet

In this notebook, we are going to implement a simplified version of the first use case: _timeseries_, or simply put, some function/variable that changes with time.

For that, the first thing we need in our countract is some way of storing the value of the variable for each point in time. In _Cairo_, we write that as:

In [116]:
%%file -a "../src/Oracle.cairo"

@storage_var
func timeseries(time: felt) -> (value: felt) {
}

Appending to ../src/Oracle.cairo


But then we also need to keep track of what the latest position in the timeseries is so we can retrieve it later on!

In [117]:
%%file -a "../src/Oracle.cairo"

@storage_var
func latest_time() -> (time: felt) {
}

Appending to ../src/Oracle.cairo


### Emitting Events for Tracking

It is also a good idea to notify offchain systems of what is happening in the Oracle, so whenever we receive new data, we could emit an event:

In [118]:
%%file -a "../src/Oracle.cairo"

@event
func value_added(time: felt, value: felt) {
}

Appending to ../src/Oracle.cairo


In this case, our event is simply a tuple `(time, value)`. In other words, whenever we have a new value added to our oracle, the contract will emit an event saying "our variable has value y at time x".

### Ownership and Access Control

Next, we need to initialize the contract during deployment! This is specially important to make sure only the owner has access to it, otherwise anyone would be able to input values into the timeseries, making it impossible to trust the data.

More advanced oracles will use a signature verification and a pool of many different submitters instead of having a single owner.

In [119]:
%%file -a "../src/Oracle.cairo"

@constructor
func constructor{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(owner: felt) {
    Ownable.initializer(owner);

    return ();
}

Appending to ../src/Oracle.cairo


### Internal Business Logic

To make things more interesting, we are going to process each value added to our function in such a way that it can only be a number between 0-100.

To do that, we are going to create a `_process_value` function that is not acessible externally. It will take a `felt` value and return a `felt` bounded between 0 and 100. The bounding process will simply be "output 100 if the input is 100 or higher".

In [120]:
%%file -a "../src/Oracle.cairo"

func _process_value{range_check_ptr}(original: felt) -> (processed: felt) {
    let (quotient, _) = unsigned_div_rem(original, 100);

    if (quotient == 0) {
        return (processed=original);
    }

    return (processed=100);
}

Appending to ../src/Oracle.cairo


### Data Ingestion

Great! But how do we add information to the oracle? We need a function for that that can be called from outside the chain! This function is going to be called `ingest()`. It will take in a value and store it associated *with the current block*, which is our measure of time.

In [121]:
%%file -a "../src/Oracle.cairo"

@external
func ingest{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(value: felt) {
    Ownable.assert_only_owner();

    let (tip) = latest_time.read();
    let (time) = get_block_number();
    
    with_attr error_message("Value already ingested at current block") {
        assert_lt(tip, time);
    }
    
    let (processed_value) = _process_value(value);
    timeseries.write(time, processed_value);
    latest_time.write(time);
    
    value_added.emit(time, processed_value);
    
    return ();
}

Appending to ../src/Oracle.cairo


### Accessing Information from Other Contracts

Finally we should provide blockchain users (e.g.: other contracts) with some form of fetching the information inside the chain:

In [122]:
%%file -a "../src/Oracle.cairo"

@view
func value_at{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}(time: felt) -> (value: felt) {
    return timeseries.read(time);
}

@view
func latest_timestamp{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}() -> (time: felt) {
    return latest_time.read();
}

@view
func latest_value{syscall_ptr: felt*, pedersen_ptr: HashBuiltin*, range_check_ptr}() -> (value: felt) {
    let (tip) = latest_time.read();
    
    return value_at(tip);
}


Appending to ../src/Oracle.cairo


Excellent! We have a full contract written to `../src/Oracle.cairo` now! We just need to test it actually implements everything correctly and then deploy & play with it!

### Browse the Full Contract Source Code

⚠️ link only available when running locally ⚠️

[../src/Oracle.cairo](/edit/src/Oracle.cairo)

# Tests

Lets check our implementation by running [Protostar][protostar].

[protostar]: https://docs.swmansion.com/protostar/

In [123]:
%%bash

cd ..
protostar test tests/test_Oracle.cairo

10:42:01 [INFO] Collected 1 suite, and 4 test cases (0.059 s)
10:42:06 [INFO] Test suites: 1 passed, 1 total
10:42:06 [INFO] Tests:       4 passed, 4 total
10:42:06 [INFO] Seed:        384626648
10:42:06 [INFO] Execution time: 6.44 s


[PASS] tests/test_Oracle.cairo test_ingest_overflowing_value (time=0.11s, steps=451, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10
[PASS] tests/test_Oracle.cairo test_ingest_edge_value (time=0.11s, steps=451, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10
[PASS] tests/test_Oracle.cairo test_ingest_values (time=0.37s, steps=858, memory_holes=40)
       pedersen_builtin=4 range_check_builtin=20
[PASS] tests/test_Oracle.cairo test_ingest_value (time=0.11s, steps=451, memory_holes=20)
       pedersen_builtin=2 range_check_builtin=10



Everything looking good! Now we just need to deploy our finished contract and play around with it!

# Compiling the Contract

In [81]:
%%bash

cd ..

protostar build

23:52:49 [INFO] Built the project successfully
23:52:49 [INFO] Execution time: 4.24 s


# Deploying the Contract

Now we can deploy our contract to `testnet`. Note that we use the wallet address we created in the first step as the owner, so replace with yours:

In [82]:
%%bash

cd ..

protostar deploy ./build/oracle.json \
                 --network testnet \
                 --inputs 0x004765409f0dd450fa303e5f7cebfb1304b2f495b5ce940da8a5aa779018a412

23:52:56 [INFO] Deploy transaction was sent.
Contract address: 0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb
Transaction hash: 0x06236f0fbd4a287194cbb953140bab23afb577429e7fa50f28b2cac733460e6e

https://goerli.voyager.online/contract/0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb
23:52:56 [INFO] Execution time: 4.19 s


# Ingesting Data

Below we call the `invoke()` function in the contract with the value `42`:

In [84]:
%%bash

source ~/cairo38_venv/bin/activate
cd ..

export STARKNET_WALLET=starkware.starknet.wallets.open_zeppelin.OpenZeppelinAccount

starknet invoke \
         --network alpha-goerli \
         --address 0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb \
         --abi build/oracle_abi.json \
         --function ingest \
         --inputs 42

Sending the transaction with max_fee: 0.000000 ETH.
Invoke transaction was sent.
Contract address: 0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb
Transaction hash: 0x69593ec4c1b92a5781c9630614708c9af8c13abc395a2536d0655f142ba429c


# Retrieving Data

After the ingestion transaction gets completed, we can verify it actually works by checking the `latest_value()`!

In [86]:
%%bash

source ~/cairo38_venv/bin/activate
cd ..

starknet call \
         --network alpha-goerli \
         --address 0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb \
         --abi build/oracle_abi.json \
         --function latest_value

42


And also the block number (i.e.: timestamp) for that value:

In [87]:
%%bash

source ~/cairo38_venv/bin/activate
cd ..

starknet call \
         --network alpha-goerli \
         --address 0x0293496496ba87bcd4c0e277bacffd7064475a26fdf009706968dd310c1429fb \
         --abi build/oracle_abi.json \
         --function latest_timestamp

345634


# Using Voyager

Note that we can also use [Voyager][voyager] to interact with the contract.

[voyager]: https://voyager.online